Вопросно-ответный поиск в промышленной отрасли

**Содержание проекта:**
* Обучим и протестируем модель для поиска ответа на вопросы в промышленной отрасли. На входе вопрос пользователя, система ищет похожий вопрос в базе вопросов с ответами и выдает ответ.
* Проведем анализ решения и альтернатив.
* Выберем лучшую модель.

In [ ]:
# установим нужные библиотеки
!pip install transformers
!pip install wikipedia
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00


In [ ]:
# загрузим нужные нам библиотеки
import numpy as np
import pandas as pd
import json

import random
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import pipeline
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from gensim.parsing.preprocessing import preprocess_string
from gensim.models import Word2Vec
import gensim.downloader

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_curve, auc

from sklearn.feature_extraction.text import*
from sklearn import preprocessing
from sklearn import metrics
from subprocess import check_output
import wikipedia as wiki

import string
import re
import pickle

import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# выведем список файлов в репозитории колаба (предварительно их туда загрузив)
print(check_output(["ls", "/content/"]).decode("utf8"))

dev-v1.1.json
kaggle.json
sample_data
train-v1.1.json



In [ ]:
# запишем функцию для чтения dev.json файла
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers', 'c_id'],
                           verbose = 1):
    if verbose:
        print("Reading the json file")
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # парсим данные на разных уровнях
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])

    #соединяем в один датафрейм
    idx = np.repeat(r['context'].values, r.qas.str.len())

    m['context'] = idx

    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

# запишем функцию для чтения train.json файла
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    if verbose:
        print("Reading the json file")
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # парсим данные на разных уровнях
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])

    #соединяем в один датафрейм
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [ ]:
# training data
input_file_path = '/content/train-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (87599, 6)
Done


In [ ]:
train.head(10)

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0
5,5733bf84d058e614000b61be,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",248,September 1876,1
6,5733bf84d058e614000b61bf,How often is Notre Dame's the Juggler published?,"As at most other universities, Notre Dame's st...",441,twice,1
7,5733bf84d058e614000b61c0,What is the daily student paper at Notre Dame ...,"As at most other universities, Notre Dame's st...",598,The Observer,1
8,5733bf84d058e614000b61bd,How many student news papers are found at Notr...,"As at most other universities, Notre Dame's st...",126,three,1
9,5733bf84d058e614000b61c1,In what year did the student paper Common Sens...,"As at most other universities, Notre Dame's st...",908,1987,1


In [ ]:
# проверим мощности для обработки модели
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
    device = 'cuda'
else:
    print('No GPU available, training on CPU.')
    device = 'cpu'

No GPU available, training on CPU.


In [ ]:
# зададим вопрос на английском языке, выведем в качестве результата 5 статей похожей тематики
k = 5
question = input("Enter the question: \n")

# ответы в виде статей будем искать на википедии
results = wiki.search(question, results=k)
print('Question:', question)
print('Pages:  ', results)

Enter the question: 
What is the name of Putin?
Question: What is the name of Putin?
Pages:   ['Vladimir Putin', 'International Criminal Court arrest warrants for Vladimir Putin and Maria Lvova-Belova', 'Russia under Vladimir Putin', 'Putinism', 'Timeline of Russian interference in the 2016 United States elections']


In [ ]:
# подберем уникальные документы для поиска нашего ответа из наших данных
documents = train[['context', 'c_id']].drop_duplicates().reset_index(drop=True)
documents

,context,c_id
0,"Architecturally, the school has a Catholic cha...",0
1,"As at most other universities, Notre Dame's st...",1
2,The university is the major seat of the Congre...,2
3,The College of Engineering was established in ...,3
4,All of Notre Dame's undergraduate students are...,4
...,...,...
18886,"Institute of Medicine, the central college of ...",18886
18887,Football and Cricket are the most popular spor...,18887
18888,The total length of roads in Nepal is recorded...,18888
18889,The main international airport serving Kathman...,18889


In [ ]:
# зададим параметры для TF-IDF
tfidf_configs = {
    'lowercase': True,
    'analyzer': 'word',
    'stop_words': 'english',
    'binary': True,
    'max_df': 0.9,
    'max_features': 10_000
}
# определим количество документов для отображения
retriever_configs = {
    'n_neighbors': 10,
    'metric': 'cosine'
}

# определим последовательность обработки
embedding = TfidfVectorizer(**tfidf_configs)
retriever = NearestNeighbors(**retriever_configs)

In [ ]:
# обучим модель для получения id документа из 'c_id'
X = embedding.fit_transform(documents['context'])
retriever.fit(X, documents['c_id'])

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
# создадим функцию обработки текста
def transform_text(vectorizer, text):
    print('Text:', text)
    vector = vectorizer.transform([text])
    vector = vectorizer.inverse_transform(vector)
    print('Vect:', vector)

In [ ]:
# сделаем векторизацию нашего вопроса
transform_text(embedding, question)

Text: What is the name of Putin?
Vect: [array([], dtype='<U18')]


In [ ]:
# предскажем наиболее похожий по контексту документ
X = embedding.transform([question])
c_id = retriever.kneighbors(X, return_distance=False)[0][0]
selected = documents.iloc[c_id]['context']

# векторизуем документ
transform_text(embedding, selected)

Text: The primary responsibility for a boy's studies lies with his House Master, but he is assisted by an additional director of studies, known as a tutor. Classes, colloquially known as "divs" (divisions), are organised on a School basis; the classrooms are separate from the houses. New school buildings have appeared for teaching purposes every decade or so since New Schools, designed by Henry Woodyer and built 1861-3. Despite the introduction of modern technology, the external appearance and locations of many of the classrooms have remained unchanged for a long time.
Vect: [array(['unchanged', 'time', 'technology', 'teaching', 'studies',
       'separate', 'schools', 'school', 'responsibility', 'remained',
       'purposes', 'primary', 'organised', 'new', 'modern', 'master',
       'long', 'locations', 'lies', 'known', 'introduction', 'houses',
       'house', 'henry', 'external', 'divisions', 'director', 'despite',
       'designed', 'decade', 'colloquially', 'classes', 'built',
   

In [ ]:
%%time
# сделаем предсказание для каждого документа по отношению к вопросу
X = embedding.transform(train['question'])
y_test = train['c_id']
y_pred = retriever.kneighbors(X, return_distance=False)

CPU times: user 29.2 s, sys: 26.6 s, total: 55.9 s
Wall time: 56.7 s


In [ ]:
# обозначим топ-лист документов для каждого вопроса
y_pred

array([[    0,  3694, 10613, ..., 17590,  6913,  6912],
       [    7,  1469,     2, ...,    29, 14201,    17],
       [   38,  1469, 14152, ...,    28,     7, 14201],
       ...,
       [18890, 18884, 18836, ..., 12302, 18837,  4200],
       [18890,  3537, 18841, ..., 16014, 18884, 10882],
       [12592, 12591, 12598, ..., 12593, 12600, 12588]])

In [ ]:
# напишем функцию для подсчета метрик качества
def top_accuracy(y_true, y_pred) -> float:
    right, count = 0, 0
    for i, y_t in enumerate(y_true):
        count += 1
        if y_t in y_pred[i]:
            right += 1
    return right / count if count > 0 else 0

In [ ]:
# выведем метрики качества для наших данных
acc = top_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')
print('Quantity:', int(acc*len(y_pred)), 'from', len(y_pred))

Accuracy: 0.7148
Quantity: 62615 from 87599


In [ ]:
# accuracy достаточно высокая

In [ ]:
# посмотрим другие решения

In [ ]:
# создаем корпус токенов
corpus = documents['context'].tolist()
corpus = [preprocess_string(t) for t in corpus]

In [ ]:
vectorizer = Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4).wv

In [ ]:
vectorizer.most_similar('putin', topn=5)

[('steamship', 0.6413528919219971),
 ('freedmen', 0.6341046094894409),
 ('ottoman', 0.6340005993843079),
 ('zunghar', 0.631178617477417),
 ('afghan', 0.6267966032028198)]

In [ ]:
def transform_text2(vectorizer, text, verbose=False):
    '''
    Transform the text in a vector[Word2Vec]
    vectorizer: sklearn.vectorizer
    text: str
    '''
    tokens = preprocess_string(text)
    words = [vectorizer[w] for w in tokens if w in vectorizer]
    if verbose:
        print('Text:', text)
        print('Vector:', [w for w in tokens if w in vectorizer])
    elif len(words):
        return np.mean(words, axis=0)
    else:
        return np.zeros((300), dtype=np.float32)

In [ ]:
transform_text2(vectorizer, question, verbose=True)

Text: What is the name of Putin?
Vector: ['putin']


In [ ]:
# обучим модель
retriever2 = NearestNeighbors(**retriever_configs)

# векторизуем документ
X = documents['context'].apply(lambda x: transform_text2(vectorizer, x)).tolist()
retriever2.fit(X, documents['c_id'])

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
%%time
# векторизуем вопрос
X = train['question'].apply(lambda x: transform_text2(vectorizer, x)).tolist()

# сделаем предсказание для каждого документа по отношению к вопросу
y_test = train['c_id']
y_pred = retriever2.kneighbors(X, return_distance=False)

CPU times: user 1min 2s, sys: 9.77 s, total: 1min 12s
Wall time: 1min 16s


In [ ]:
y_pred

array([[ 2637,  6934, 18125, ...,  7082,  2014,  7052],
       [ 2424,  7290,  4816, ...,  4809,  6981, 11078],
       [10619, 10636,     0, ...,  3010,  9839,  5560],
       ...,
       [ 7933,  9491, 14938, ...,  5522,  9497, 10401],
       [ 2844,  1623, 18538, ...,  6350,   563,  1643],
       [15409, 13316, 11381, ..., 12150, 18548, 13279]])

In [ ]:
acc = top_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')
print('Quantity:', int(acc*len(y_pred)), 'from', len(y_pred))

Accuracy: 0.1200
Quantity: 10511 from 87599


In [ ]:
# accuracy для word2vec ниже, чем для tf-idf

Посмотрим другие модели, уже предобученные, но с указанием текста, из которого нужно вытащить ответ

In [ ]:
# определим модель и токенизатор для использования в модели - используем предобученный Bert для ответов на вопросы
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

text = "Vladimir Vladimirovich Putin[c] (born 7 October 1952) is a Russian politician and former intelligence officer who has been President of Russia since 2012. Putin has held continuous positions as president or prime minister since 1999:[d] as prime minister from 1999 to 2000 and from 2008 to 2012, and as president from 2000 to 2008 and since 2012. Putin worked as a KGB foreign intelligence officer for 16 years, rising to the rank of lieutenant colonel before resigning in 1991 to begin a political career in Saint Petersburg. In 1996, he moved to Moscow to join the administration of President Boris Yeltsin. He briefly served as the director of the Federal Security Service (FSB) and then as secretary of the Security Council of Russia before being appointed prime minister in August 1999. Following Yeltsin's resignation, Putin became acting president and, in less than four months, was elected to his first term as president. He was subsequently reelected in 2004. Due to constitutional limitations of two consecutive presidential terms, Putin served as prime minister again from 2008 to 2012 under Dmitry Medvedev. He returned to the presidency in 2012, following an election marked by allegations of fraud and protests, and was reelected in 2018. In April 2021, after a referendum, he signed into law constitutional amendments that included one allowing him to run for reelection twice more, potentially extending his presidency to 2036.[8][9]"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)


'vladimir vladimirovich putin'

In [ ]:
outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

Вывод:

Предобученная модель DistilBert выдаёт готовый ответ за счет моделирование языка по маске (BERT делал это с помощью MLM и Next Sentence Prediction) и обучения с использованием тройного лосса.

*(P.S. модель Bert не рассматривалсь, т.к. DistilBERT нацелен на оптимизацию обучения за счет уменьшения размера и увеличения скорости BERT. Также, DistilBERT весит на 40% меньше, чем оригинальная BERT-модель, она на 60% быстрее ее и сохраняет 97% ее функциональности)*

Мы посмотрели применение TF-IDF и Word2Vec, где accuracy для TF-IDF была намного выше.

Тогда как минус TF-IDF в том, что он выдает похожие по контексту: алгоритм способен вычислить сходство только между вопросами и документами, в которых представлены одни и те же слова, поэтому он не может улавливать синонимы; и не может понять контекст вопроса или значение слов.

Для получения готового ответа выгоднее использовать предобученную модель DistilBert. Из возможных дальнейших шагов докрутки - это настройка ответа имён соьсвтенных с заглавной буквы, как и в источнике информации, откуда берется ответ.